In [17]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/CoronaTweetsSentimentAnalysis.csv')
data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
1,2,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
2,3,44959,NaN,03-03-2020,Voting in the age of #coronavirus = hand sanit...,Positive
3,4,44963,"Boksburg, South Africa",04-03-2020,Best quality couches at unbelievably low price...,Positive
4,5,44967,"Washington, DC",04-03-2020,While we were busy watching election returns a...,Positive


In [18]:
data['Location'].value_counts()

United States             42
London, England           25
Washington, DC            22
New York, NY              20
Los Angeles, CA           16
                          ..
Paris, France              1
FB.me/EnglishInRomania     1
West Palm Beach, FL        1
310                        1
Farmington, NM             1
Name: Location, Length: 1019, dtype: int64

In [19]:
data['Sentiment'].unique()

array(['Positive', 'Negative'], dtype=object)

In [20]:
import string
import re, nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

nltk.download('punkt')
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

def cleantweet(text):
  tokens= nltk.word_tokenize(re.sub("[^a-zA-Z]", " ",text))
  tokens = [token.lower() for token in tokens]
  return ' '.join(tokens[2:])

                 

def textpreprocess(msg):
  nopunc =[char for char in msg if char not in string.punctuation]
  nopunc=''.join(nopunc)
  return ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# removing neutral tweets
data['aftercleaning_tweets'] = data['OriginalTweet'].apply(cleantweet)
data['Sentiment'] = data['Sentiment'].apply(lambda x: 1 if x =='Positive' else 0)
data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,aftercleaning_tweets
0,1,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,1,couldn t find hand sanitizer at fred meyer i t...
1,2,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,0,hits newyork city as anxious shoppers stock up...
2,3,44959,NaN,03-03-2020,Voting in the age of #coronavirus = hand sanit...,1,the age of coronavirus hand sanitizer supertue...
3,4,44963,"Boksburg, South Africa",04-03-2020,Best quality couches at unbelievably low price...,1,couches at unbelievably low prices available t...
4,5,44967,"Washington, DC",04-03-2020,While we were busy watching election returns a...,1,were busy watching election returns and bracin...


In [23]:
data['aftercleaning_tweets'] = data['aftercleaning_tweets'].apply(textpreprocess)
data.reset_index(drop=True, inplace = True)
data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,aftercleaning_tweets
0,1,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,1,find hand sanitizer fred meyer turned amazon p...
1,2,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,0,hits newyork city anxious shoppers stock food ...
2,3,44959,NaN,03-03-2020,Voting in the age of #coronavirus = hand sanit...,1,age coronavirus hand sanitizer supertuesday ht...
3,4,44963,"Boksburg, South Africa",04-03-2020,Best quality couches at unbelievably low price...,1,couches unbelievably low prices available orde...
4,5,44967,"Washington, DC",04-03-2020,While we were busy watching election returns a...,1,busy watching election returns bracing covid o...


In [24]:
data['Sentiment'].unique()

array([1, 0])

In [28]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# Creating object of TF-IDF vectorizer
vectorizer = TfidfVectorizer(use_idf=True, lowercase=True)
X_tf_idf= vectorizer.fit_transform(data.aftercleaning_tweets)
x_train, x_test, y_train, y_test = train_test_split(X_tf_idf, data['Sentiment'], random_state=42)

In [30]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
model=clf.fit(x_train.toarray(), y_train)
predicted_class=model.predict(x_test.toarray())
predicted_class_train=model.predict(x_train.toarray())
test_probs = model.predict_proba(x_test.toarray())
test_probs = test_probs[:, 1]
yhat = model.predict(x_test.toarray())
print('Train confusion matrix is: ',)
print(confusion_matrix(y_train, predicted_class_train))
print()
print('Test confusion matrix is: ')
print(confusion_matrix(y_test, predicted_class))
print()
print(classification_report(y_test,predicted_class)) 
print() 
train_accuracy = accuracy_score(y_train,predicted_class_train)
test_accuracy = accuracy_score(y_test,predicted_class)

print("Train accuracy score: ", train_accuracy)
print("Test accuracy score: ",test_accuracy )

Train confusion matrix is: 
[[739  36]
 [  0 716]]

Test confusion matrix is: 
[[115 151]
 [ 92 139]]

              precision    recall  f1-score   support

           0       0.56      0.43      0.49       266
           1       0.48      0.60      0.53       231

    accuracy                           0.51       497
   macro avg       0.52      0.52      0.51       497
weighted avg       0.52      0.51      0.51       497


Train accuracy score:  0.9758551307847082
Test accuracy score:  0.5110663983903421
